# Algoritmos de Aprendizaje no Supervisado

## I.	Clustering

## II.	Reglas de Asociación

In [5]:
import pandas as pd
from apyori import apriori
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

# Cargar los datos
movies = pd.read_csv('movies.csv', encoding='cp1252')

In [7]:
movies.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,5,4000000,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,United States of America,4257354.0,98,False,...,20.880,1995-12-09,5.7,2077,2,2,1,25,15,9
1,6,21000000,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,12136938.0,110,False,...,9.596,1993-10-15,6.5,223,3,3,2,15,3,9
2,11,11000000,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,United States of America,775398007.0,121,NaN,...,100.003,1977-05-25,8.2,16598,3,2,1,105,5,62
3,12,94000000,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,United States of America,940335536.0,100,NaN,...,134.435,2003-05-30,7.8,15928,2,1,1,24,5,18
4,13,55000000,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,United States of America,677387716.0,142,False,...,58.751,1994-07-06,8.5,22045,3,2,1,76,18,48


In [14]:
print(movies.shape)
print(movies['productionCompanyCountry'].value_counts())


(10000, 27)
productionCompanyCountry
US                          997
US|US                       792
US|US|US                    452
|US                         292
US|                         265
                           ... 
|GB|US|US|GB|CH||GB|GB        1
|JP|JP|JP|JP|JP|JP|JP|JP      1
US|US|JP|GB                   1
US|US|GB|GB|US|US             1
US|US|CA|US                   1
Name: count, Length: 2354, dtype: int64


In [ ]:
numericas_selector = make_column_selector(dtype_exclude=object)
categoricas_selector = make_column_selector(dtype_include=object)
numericas_0 = numericas_selector(movies)
categoricas_0 = categoricas_selector(movies)

print(numericas_0)

print(categoricas_0)

# Eliminamos columnas que no aportan valor al análisis
'''
asda


columnas_a_eliminar = ['id', 'homePage', 'actorsPopularity', 'actorsCharacter', 
                      'originalTitle', 'title', 'releaseDate']
movies = movies.drop(columnas_a_eliminar, axis=1)
'''

numericas = ['budget', 'revenue', 'runtime', 'popularity', 'voteAvg', 'voteCount',
             'genresAmount', 'productionCoAmount', 'productionCountriesAmount', 'actorsAmount']
categoricas = ['genres', 'productionCompany', 'productionCompanyCountry', 
               'productionCountry', 'video', 'director', 'actors', 
               'originalLanguage', 'castWomenAmount', 'castMenAmount']

print("Forma del dataset después de eliminar columnas:", movies.shape)


['budget', 'revenue', 'runtime', 'popularity', 'voteAvg', 'voteCount', 'genresAmount', 'productionCoAmount', 'productionCountriesAmount', 'actorsAmount']
['genres', 'productionCompany', 'productionCompanyCountry', 'productionCountry', 'video', 'director', 'actors', 'originalLanguage', 'castWomenAmount', 'castMenAmount']


KeyError: "['id', 'homePage', 'actorsPopularity', 'actorsCharacter', 'originalTitle', 'title', 'releaseDate'] not found in axis"

In [20]:
# Paso 2: Manejo de valores nulos en variables numéricas
for numerica in numericas:
    movies[numerica] = movies[numerica].fillna(movies[numerica].median())
    print(f"Valores nulos en {numerica} después de imputación:", 
          movies[numerica].isnull().sum())

# Verificamos los valores únicos en algunas variables categóricas clave
print("\nValores únicos en video:", movies['video'].unique())
print("\nValores únicos en originalLanguage:", movies['originalLanguage'].unique())

# Para las variables categóricas, reemplazamos nulos con 'Unknown'
for categorica in categoricas:
    movies[categorica] = movies[categorica].fillna('Unknown')

KeyError: 'id'

## III.	Análisis de Componentes Principales